In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import r2_score

# Đọc dữ liệu từ file CSV và đổi tên cột
data = pd.read_csv("./Dataset/STB.csv")
data.rename(columns={"trunc_time":"Date","open_price":"open","high_price":"high","low_price":"low","close_price":"Close"}, inplace=True)

# Lấy dữ liệu chỉ với cột 'Date' và 'Close'
closedf = data[['Date', 'Close']]

# Chuẩn hóa dữ liệu 'Close' trong khoảng từ 0 đến 1
scaler = MinMaxScaler(feature_range=(0, 1))
closedf['Close'] = scaler.fit_transform(closedf['Close'].values.reshape(-1, 1))

# Tạo tập dữ liệu huấn luyện và kiểm tra
time_step = 10
training_size = int(len(closedf) * 0.8)
test_size = len(closedf) - training_size
train_data, test_data = closedf.iloc[:training_size], closedf.iloc[training_size:]

# Hàm tạo dữ liệu dạng chuỗi cho LSTM
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset['Close'].values[i:(i + time_step)]
        dataX.append(a)
        dataY.append(dataset['Close'].values[i + time_step])
    return np.array(dataX), np.array(dataY)

# Tạo dữ liệu huấn luyện và kiểm tra cho LSTM
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape lại dữ liệu cho phù hợp với đầu vào LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Xây dựng mô hình LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')

# Huấn luyện mô hình LSTM
lstm_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Dự đoán giá cổ phiếu
train_predict = lstm_model.predict(X_train)
test_predict = lstm_model.predict(X_test)

# Đưa giá trị dự đoán về dạng ban đầu
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])

# In ra kết quả dự đoán và thực tế
print("Train Mean Absolute Error:", np.mean(np.abs(y_train - train_predict)))
print("Train Root Mean Squared Error:", np.sqrt(np.mean(np.square(y_train - train_predict))))
print("Test Mean Absolute Error:", np.mean(np.abs(y_test - test_predict)))
print("Test Root Mean Squared Error:", np.sqrt(np.mean(np.square(y_test - test_predict))))


/tmp/ipykernel_72678/254124276.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closedf['Close'] = scaler.fit_transform(closedf['Close'].values.reshape(-1, 1))
2023-06-09 10:47:11.355736: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-09 10:47:11.357403: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): I

Epoch 1/20


2023-06-09 10:47:11.742627: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-09 10:47:11.744014: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-09 10:47:11.745113: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

43/43 [==============================] - ETA: 0s - loss: 0.0058

2023-06-09 10:47:15.103842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-09 10:47:15.105026: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-09 10:47:15.105996: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

43/43 [==============================] - 4s 25ms/step - loss: 0.0058 - val_loss: 0.0084
Epoch 2/20
43/43 [==============================] - 0s 10ms/step - loss: 7.1332e-04 - val_loss: 0.0050
Epoch 3/20
43/43 [==============================] - 0s 9ms/step - loss: 6.3844e-04 - val_loss: 0.0025
Epoch 4/20
43/43 [==============================] - 0s 8ms/step - loss: 5.9116e-04 - val_loss: 0.0025
Epoch 5/20
43/43 [==============================] - 0s 9ms/step - loss: 5.9017e-04 - val_loss: 0.0024
Epoch 6/20
43/43 [==============================] - 0s 8ms/step - loss: 5.6793e-04 - val_loss: 0.0039
Epoch 7/20
43/43 [==============================] - 0s 9ms/step - loss: 5.6731e-04 - val_loss: 0.0032
Epoch 8/20
43/43 [==============================] - 0s 10ms/step - loss: 5.9890e-04 - val_loss: 0.0042
Epoch 9/20
43/43 [==============================] - 0s 11ms/step - loss: 5.4806e-04 - val_loss: 0.0029
Epoch 10/20
43/43 [==============================] - 0s 9ms/step - loss: 5.3797e-04 - val_los

2023-06-09 10:47:23.670474: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-09 10:47:23.672270: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-09 10:47:23.673757: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

 1/43 [..............................] - ETA: 27s

2023-06-09 10:47:23.875863: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_1_grad/concat/split_1/split_dim' with dtype int32
	 [[{{node gradients/split_1_grad/concat/split_1/split_dim}}]]


11/11 [==============================] - 0s 4ms/step
Train Mean Absolute Error: 3666.207290727167
Train Root Mean Squared Error: 5891.97483115925
Test Mean Absolute Error: 5992.125519412035
Test Root Mean Squared Error: 7425.411413207406
